In [ ]:
# Try to build you code based on this one, so it will be easier for us to intergate all the codes into one file
# Create a function for each plot that you want to call
# Replace the screenshot in final report to the function you want to call 

In [10]:
import pandas as pd
import numpy as np
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Import data
# This cell should applies to the code of all of us
data_path = "./data/"
subscriptions = pd.read_csv(data_path + 'subscriptionsdataFINAL.csv') 
visitation_part1 = pd.read_csv(data_path + 'visitationdataPART1.csv') 
visitation_part2 = pd.read_csv(data_path + 'visitationdataPART2.csv') #convert the join and end date to_date
subscriptions['Join Date'] =  pd.to_datetime(subscriptions['Join Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions['Join_year'] = pd.DatetimeIndex(subscriptions['Join Date']).year
subscriptions['End Date'] =  pd.to_datetime(subscriptions['End Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions.dropna()
subscriptions.head() 


In [13]:
# Create the seperate dataset to avoid any variable name conflict 
subscriptions_Qian =  subscriptions.copy()
visitation_part1_Qian = visitation_part1.copy() 
visitation_part2_Qian = visitation_part2.copy()  

In [ ]:
# Data Processing - Qian
df = subscriptions_Qian.copy()
numbers_club = pd.DataFrame(df.groupby('Join_year')['Club'].nunique())
numbers_club['Growth']=numbers_club['Club'].pct_change()
